In [0]:
#import findspark
#findspark.init('/spark/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max, desc
from pyspark.sql.functions import grouping, explode, array_contains, expr



#spark = SparkSession.builder.appName("movielens").getOrCreate()

# Consultas sobre Movielens

## Schema de Movielens

![Schema](movielens_schema.png)

## ¿Cuál es la distribución de la clasificación de las películas? 

Proceso:

1. Cargar la tabla ratings
1. Cargar la tabla movies
1. Unir ambas tablas
1. Agrupar por calificación
1. Contar las películas


### Cargar las tablas

In [0]:
# Tabla Ratings
ratings_schema  = StructType(fields=[
    StructField("userId",IntegerType(),True), 
    StructField("movieId",IntegerType(),True),
    StructField("rating",DecimalType(precision=2,scale=1),True),
    StructField("timestamp",LongType(),True)
])
ratingsDf = spark.read\
    .option("header", True)\
    .option("dateFormat", "yyyyMMdd")\
    .schema(ratings_schema)\
    .csv("/Volumes/big_data_ii_2025/spark_examples/spark_data/ratings.csv")\
    .withColumn(\
            "date",\
            from_unixtime("timestamp", "yyyyMMdd"))\
                .drop('timestamp')

# Tabla Movies
movies_schema  = StructType(fields=[
    StructField("movieId",IntegerType(),True), 
    StructField("title",StringType(),True),
    StructField("genres",StringType(),True)
])

moviesDf = spark.read\
    .option("header", True)\
    .schema(movies_schema)\
    .csv("/Volumes/big_data_ii_2025/spark_examples/spark_data/movies.csv")

moviesDf = moviesDf.withColumn("genresSplit", split(moviesDf["genres"],"\|"))\
                        .drop('genres').withColumnRenamed("genresSplit","genres")\
                            .withColumn(\
                                "year",\
                                regexp_extract(\
                                           moviesDf["title"],\
                                           "^.+\(([0-9]+)\)$",\
                                           1)\
                                .try_cast(IntegerType()))\
                            .withColumn(\
                            "title_temp",\
                            regexp_extract(\
                                           moviesDf["title"],\
                                           "^(.+?) \([0-9]+\)$",\
                                           1))\
                            .drop('title')\
                        .withColumnRenamed("title_temp","title")

### Unir Ambas tablas

In [0]:
movie_ratingsDF = ratingsDf.join(moviesDf,on="movieId",how="inner")
movie_ratingsDF.show(3)

### Filtros

In [0]:
movie_ratingsDF.filter("movieId = 356").select(["rating","genres"]).show(3)

In [0]:
movie_ratingsDF.filter(col('genres').getItem(0) == 'Drama').select(["rating","genres"]).show(3)

In [0]:
movie_ratingsDF.filter(array_contains(movie_ratingsDF.genres,'Drama')).select(["rating","genres"]).show(10)

# Funciones de Ventana en PySpark

## ¿Qué son las Funciones de Ventana?

Las funciones de ventana en PySpark permiten realizar cálculos avanzados sobre un conjunto específico de filas relacionadas, conocidas como "ventanas", sin la necesidad de agrupar los datos en su totalidad. Estas funciones son muy útiles para realizar análisis complejos y obtener insights detallados de los datos, manteniendo el contexto de cada fila.

### Características Principales de las Funciones de Ventana

1. **Cálculos sin Agrupamiento Completo:**
   - Las funciones de ventana permiten realizar operaciones como promedios, sumas, y rankings dentro de una partición de datos sin agrupar completamente los datos.
   
2. **Contexto Manteniendo el Orden:**
   - Mantienen el contexto de cada fila dentro de una partición, permitiendo realizar cálculos acumulativos y de comparación.
   
3. **Ventanas Definidas Dinámicamente:**
   - Las ventanas pueden ser definidas basándose en particiones y órdenes específicos, lo que permite una gran flexibilidad en el análisis de datos.

### Tipos Comunes de Funciones de Ventana

- **Agregaciones:**
  - `avg()`, `sum()`, `min()`, `max()`: Calculan el promedio, suma, mínimo y máximo dentro de la ventana.
  
- **Funciones de Rango y Ranking:**
  - `row_number()`, `rank()`, `dense_rank()`: Asignan números de fila y rangos dentro de la ventana.

- **Funciones de Desplazamiento:**
  - `lag()`, `lead()`: Permiten acceder a las filas anteriores o siguientes dentro de la ventana.

### Ejemplo de Uso

Considere un DataFrame de calificaciones de películas, donde queremos calcular el promedio de calificaciones por usuario, el ranking de calificaciones por género y año, y el cambio de calificación respecto a la calificación anterior dada por el mismo usuario. Las funciones de ventana nos permiten realizar estos cálculos de manera eficiente y mantener el contexto de cada calificación individual.

Con las funciones de ventana, podemos transformar y analizar los datos de manera avanzada, obteniendo insights que serían difíciles de lograr con simples agregaciones o transformaciones básicas.



### Calcular el Promedio de Calificaciones por Usuario

En este ejemplo, calcularemos el promedio de calificaciones de cada usuario para todas las películas que han calificado. Usaremos la función de ventana avg para realizar este cálculo.



In [0]:
from pyspark.sql.window import Window

# Suponiendo que movie_ratingsDF ya está cargado
windowSpec = Window.partitionBy("userId")

# Calcular el promedio de calificaciones por usuario
average_ratingsDF = movie_ratingsDF\
                    .withColumn(\
                                "average_rating", \
                                avg("rating")\
                                .over(windowSpec))

average_ratingsDF.printSchema()


In [0]:
average_ratingsDF.show()

### Calcular el Ranking de Calificaciones por Género y Año

En este ejemplo, calcularemos el ranking de las calificaciones de las películas dentro de cada género y año. Utilizaremos la función de ventana rank para determinar la posición de cada película en términos de calificación.



In [0]:
from pyspark.sql.functions import rank

explodedDF = movie_ratingsDF.withColumn("genre", explode("genres"))

windowSpec = Window.partitionBy("genre", "year").orderBy(col("rating").desc())

ranked_ratingsDF = explodedDF.withColumn("rank", rank().over(windowSpec))

ranked_ratingsDF.show()


### Calcular el Cambio de Calificación respecto a la Calificación Anterior por Usuario

En este ejemplo, calcularemos el cambio de calificación de una película respecto a la calificación anterior dada por el mismo usuario. Usaremos la función de ventana lag para acceder a la calificación anterior y calcular la diferencia.

In [0]:
from pyspark.sql.functions import lag


# Suponiendo que movie_ratingsDF ya está cargado
windowSpec = Window.partitionBy("userId").orderBy("date")

# Calcular el cambio de calificación respecto a la calificación anterior por usuario
change_ratingsDF = movie_ratingsDF.withColumn("previous_rating", lag("rating", 1).over(windowSpec)) \
                                  .withColumn("rating_change", col("rating") - col("previous_rating"))

change_ratingsDF.show()


Ordenamos la lista para que se vea mejor:

## ¿Cuál es la película con la mayor cantidad de reseñas? 

In [0]:
movie_ratingsDF_grouped_by_movieID = movie_ratingsDF.groupBy("title").count().orderBy(desc("count"))
movie_ratingsDF_grouped_by_movieID.show(1)


In [0]:
movie_ratingsDF.printSchema()

In [0]:
movie_ratingsDF.show(2)